<center>
    <h1><b>Hyper Parameter Tunning & Choosing The Best Model</b></h1>
</center>

This is the process of choosing optimum parameter

Cross fold validation is done by dividing data samples into n number of folds e.g 5 folds (i.e dividing the data into different parts or samples, each sample is then divided into 5 parts, one of this part will be used as test sample). Iteration is done on the folds and individual scores are combined to ascertain the final score.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn import svm, datasets

%matplotlib inline

In [2]:
iris = datasets.load_iris()

In [3]:
df = pd.DataFrame(iris.data, columns = iris.feature_names)
df['flower'] = iris.target
df.head(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


In [4]:
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])

In [5]:
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


In [6]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size = 0.3, random_state = 0)

In [7]:
model = svm.SVC(kernel = 'rbf', C =30, gamma = 'auto')
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9777777777777777

### Cross Fold Validation

In [8]:
cross_val_score(svm.SVC(kernel = 'linear', C=10, gamma='auto'), iris.data, iris.target, cv=5)  # cv means cross fold

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [9]:
cross_val_score(svm.SVC(kernel='rbf', C = 10, gamma = 'auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [10]:
cross_val_score(svm.SVC(kernel='rbf', C=20, gamma = 'auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

#### Runing the iterations with for loop instead of manual

In [11]:
kernels = ['rbf', 'linear']
C = [1, 10, 20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel = kval, C= cval, gamma = 'auto'), iris.data, iris.target, cv = 5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)
avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

### Using GridSearchCV API Instead of for loop

In [12]:
# Define classifier
clf = GridSearchCV(svm.SVC(gamma ='auto'), {
    'C': [1, 10, 20], 'kernel': ['rbf', 'linear']
}, cv = 5, return_train_score = False)

In [13]:
# Building the model
clf.fit(iris.data, iris.target)

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 10, 20], 'kernel': ['rbf', 'linear']})

In [14]:
# printing the result
clf.cv_results_

{'mean_fit_time': array([0.00219717, 0.00199971, 0.00179977, 0.00219922, 0.00260367,
        0.00179543]),
 'std_fit_time': array([0.00040188, 0.00063339, 0.00040052, 0.00097966, 0.00119562,
        0.00074118]),
 'mean_score_time': array([0.0012023 , 0.00119791, 0.00179844, 0.00119967, 0.0016027 ,
        0.00119982]),
 'std_score_time': array([0.00039929, 0.0003968 , 0.00097905, 0.00039961, 0.00080805,
        0.00040026]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [15]:
# Importing the result into dataframe
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002197,0.000402,0.001202,0.000399,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.002000,0.000633,0.001198,0.000397,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001800,0.000401,0.001798,0.000979,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.002199,0.000980,0.001200,0.000400,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.002604,0.001196,0.001603,0.000808,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.001795,0.000741,0.001200,0.000400,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [16]:
df = df[['param_C', 'param_kernel', 'params','mean_test_score']]
df

,param_C,param_kernel,params,mean_test_score
0,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.980000
1,1,linear,"{'C': 1, 'kernel': 'linear'}",0.980000
2,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.980000
3,10,linear,"{'C': 10, 'kernel': 'linear'}",0.973333
4,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667
5,20,linear,"{'C': 20, 'kernel': 'linear'}",0.966667


In [17]:
dir(clf)  # Seeing all the properties of clf object

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_fe

In [18]:
# To know the best score
clf.best_score_

0.9800000000000001

In [19]:
# To know the best parameter
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

# RandomizedSearchCV

This is used because iterating with many parameters on large dataset might be costly. This approach will try random composition of parameters value

In [20]:
rs = RandomizedSearchCV(svm.SVC(gamma = 'auto'), {
    'C': [1, 10, 20], 'kernel': ['rbf', 'linear']
}, cv = 5, return_train_score = False, n_iter = 2, random_state = 0)  # the number of iterations could be increased above 2

In [21]:
# Building the randomizedsearchcv model
rs.fit(iris.data, iris.target)

RandomizedSearchCV(cv=5, estimator=SVC(gamma='auto'), n_iter=2,
                   param_distributions={'C': [1, 10, 20],
                                        'kernel': ['rbf', 'linear']},
                   random_state=0)

In [22]:
df1 = pd.DataFrame(rs.cv_results_)[['param_C', 'param_kernel', 'params','mean_test_score']]
df1

,param_C,param_kernel,params,mean_test_score
0,20,linear,"{'kernel': 'linear', 'C': 20}",0.966667
1,10,rbf,"{'kernel': 'rbf', 'C': 10}",0.980000


## Selecting the Best Model

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [24]:
# Defining parameter grid
model_params = {
    'svm': {
        'model': svm.SVC(gamma ='auto'), 
        'params': {
            'C': [1, 10, 20],
            'kernel': ['rbf', 'linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [1, 5, 10]
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver = 'liblinear', multi_class ='auto'),
        'params': {
            'C': [1, 5, 10]
        }
    }
}

In [25]:
scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv = 5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

In [26]:
scores

[{'model': 'svm',
  'best_score': 0.9800000000000001,
  'best_params': {'C': 1, 'kernel': 'rbf'}},
 {'model': 'random_forest',
  'best_score': 0.96,
  'best_params': {'n_estimators': 10}},
 {'model': 'logistic_regression',
  'best_score': 0.9666666666666668,
  'best_params': {'C': 5}}]

In [27]:
df2 = pd.DataFrame(scores)
df2

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}


In [28]:
df2['score_percent'] = (df2["best_score"] * 100).round(2)
df2

,model,best_score,best_params,score_percent
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}",98.00
1,random_forest,0.960000,{'n_estimators': 10},96.00
2,logistic_regression,0.966667,{'C': 5},96.67
